This notebook only shows the results for the first 3-day period namely 27th Feb, 28th Feb and 1st March. The code for the later 3-day period is the same.

###Loading Libraries

In [1]:
!pip install flair
from flair.models import TextClassifier
from flair.data import Sentence
model = TextClassifier.load('en-sentiment')

     |████████████████████████████████| 401 kB 5.0 MB/s 
     |████████████████████████████████| 48 kB 1.6 MB/s 
     |████████████████████████████████| 981 kB 7.4 MB/s 
     |████████████████████████████████| 1.6 MB 32.5 MB/s 
     |████████████████████████████████| 46 kB 1.4 MB/s 
     |████████████████████████████████| 1.2 MB 46.0 MB/s 
     |████████████████████████████████| 77 kB 4.1 MB/s 
     |████████████████████████████████| 4.0 MB 35.7 MB/s 
     |████████████████████████████████| 19.7 MB 74.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 788 kB 22.8 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 199 kB 54.9 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.2 MB/s 
     |████████████████████████████████| 880 kB 41.2 MB/s 
     |█████████

100%|██████████| 265512723/265512723 [00:11<00:00, 24133064.76B/s]

2022-05-07 15:19:38,970 copying /tmp/tmptpj8f3le to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2022-05-07 15:19:39,557 removing temp file /tmp/tmptpj8f3le
2022-05-07 15:19:39,610 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [18]:
!pip install swifter

     |████████████████████████████████| 647 kB 5.0 MB/s 
     |████████████████████████████████| 280 kB 37.4 MB/s 
     |████████████████████████████████| 136 kB 69.5 MB/s 
  Created wheel for swifter: filename=swifter-1.1.3-py3-none-any.whl size=13078 sha256=1c3bb7d2896bf2f2af96adae8690d1d46ae0f192fe623aa7c8c1384951264f15
  Stored in directory: /root/.cache/pip/wheels/e8/1a/1b/46aa7b0fac16d3ce1bccaac14bfe4fad972642217ae78d8160
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [7]:
import numpy as np
import pandas as pd
import re
import swifter
import string

###Loading and Sampling Dataset

In [8]:
csv_collection = ["/content/UkraineCombinedTweetsDeduped20220227-131611.csv.gzip","/content/UkraineCombinedTweetsDeduped_FEB27.csv.gzip","/content/UkraineCombinedTweetsDeduped_FEB28_part1.csv.gzip","/content/UkraineCombinedTweetsDeduped_FEB28_part2.csv.gzip","/content/UkraineCombinedTweetsDeduped_MAR01.csv.gzip"]
df = pd.read_csv(csv_collection[0], compression = 'gzip')
for i in range(1,len(csv_collection)):
    tmp = pd.read_csv(csv_collection[i], compression = 'gzip')
    df = pd.concat([df, tmp], axis=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
df = df.sample(frac=0.3, random_state=1)

### Preprocessing Dataset 

In [10]:
def clean_text(preprocess_text):
  
    preprocess_text = preprocess_text.lower() #text to lower

    preprocess_text = re.sub(r'\[.*?\]', '', preprocess_text)  #remove text in square bracket
   
    preprocess_text = re.sub(r'[%s]' % re.escape(string.punctuation), '', preprocess_text) #remove punctuation 

    preprocess_text = re.sub(r'\w*\d\w*', '', preprocess_text) #remove words containing numbers
  
    preprocess_text = re.sub(r'http\S+', '', preprocess_text) #remove links 
  
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    
    preprocess_text = regrex_pattern.sub(r'', preprocess_text)
    return preprocess_text

In [11]:
df_en = df[df.language == 'en'].drop('language', axis=1)

In [12]:
tweetsdf = pd.DataFrame(df_en.text.apply(lambda x: clean_text(x)))

In [13]:
df_en['processed_text'] = tweetsdf 

In [14]:
sentiment_df = df_en[['tweetid', 'processed_text', 'hashtags']] 

Function to add a column for "Positive" and "Negative" Label and Sentiment Score

In [15]:
def flair_prediction(x):

    sentence = Sentence(x)
    
    try:        
        model.predict(sentence)
        result = sentence.labels[0]


        sentiment = str(result).replace("(",",").replace(")","").split(",")
        label = str(sentiment[0])
        score = float(sentiment[1].strip())
        
        
        if "POSITIVE" in label:
            return "POSITIVE", score
        elif "NEGATIVE" in label:
            return "NEGATIVE", score 
        else:
            return "NEUTRAL", 0.00
    except Exception:
        print(sentence)
        pass
    
    return "ERROR",0.00

In [16]:
sentiment_df['Sentiment'] = ""
sentiment_df['Sentiment_Score'] = np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
sentiment_df

,tweetid,processed_text,hashtags,Sentiment,Sentiment_Score
264745,1496970415285149703,from covid straight to worldwariii \nfrom ma...,"[{'text': 'worldwar3', 'indices': [41, 51]}, {...",,NaN
880332,1497499204486410241,something tells me this nypost coverstory is b...,"[{'text': 'coverstory', 'indices': [49, 60]}, ...",,NaN
994418,1497597933495087105,i planned to plant tulips and daffodils on my ...,[],,NaN
1000234,1497601698709356545,stop the lies stop the war\n\nthe americans re...,"[{'text': 'Russian', 'indices': [109, 117]}]",,NaN
588378,1497248264437080069,this piece about what it is like to live in uk...,"[{'text': 'Ukraine', 'indices': [44, 52]}]",,NaN
...,...,...,...,...,...
1122049,1497707586673647621,flash ⚡️in russian belgorod contract soldiers...,"[{'text': 'FLASH', 'indices': [20, 26]}]",,NaN
1134308,1497717641552748546,the anonymous collective has intercepted russi...,"[{'text': 'Anonymous', 'indices': [21, 31]}, {...",,NaN
357401,1497049451613102080,kharkiv residents spending the night in the ci...,"[{'text': 'Russia', 'indices': [111, 118]}, {'...",,NaN
296973,1496997017733660672,radio talk between russian battleship and ukra...,[],,NaN


### Predicting on Sample Tweets

In [19]:
sentiment_df["Sentiment"],sentiment_df["Sentiment_Score"] =  zip(*sentiment_df["processed_text"].swifter.apply(flair_prediction))

/usr/local/lib/python3.7/dist-packages/swifter/swifter.py:34: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, "


Pandas Apply:   0%|          | 0/285638 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
sentiment_df

,tweetid,processed_text,hashtags,Sentiment,Sentiment_Score
264745,1496970415285149703,from covid straight to worldwariii \nfrom ma...,"[{'text': 'worldwar3', 'indices': [41, 51]}, {...",POSITIVE,0.9895
880332,1497499204486410241,something tells me this nypost coverstory is b...,"[{'text': 'coverstory', 'indices': [49, 60]}, ...",NEGATIVE,0.9992
994418,1497597933495087105,i planned to plant tulips and daffodils on my ...,[],POSITIVE,0.9313
1000234,1497601698709356545,stop the lies stop the war\n\nthe americans re...,"[{'text': 'Russian', 'indices': [109, 117]}]",NEGATIVE,0.9980
588378,1497248264437080069,this piece about what it is like to live in uk...,"[{'text': 'Ukraine', 'indices': [44, 52]}]",POSITIVE,0.9937
...,...,...,...,...,...
1122049,1497707586673647621,flash ⚡️in russian belgorod contract soldiers...,"[{'text': 'FLASH', 'indices': [20, 26]}]",NEGATIVE,0.9982
1134308,1497717641552748546,the anonymous collective has intercepted russi...,"[{'text': 'Anonymous', 'indices': [21, 31]}, {...",NEGATIVE,0.9986
357401,1497049451613102080,kharkiv residents spending the night in the ci...,"[{'text': 'Russia', 'indices': [111, 118]}, {'...",NEGATIVE,0.9785
296973,1496997017733660672,radio talk between russian battleship and ukra...,[],NEGATIVE,0.9308


### Converting to CSV Files

In [21]:
from google.colab import files

In [22]:
sentiment_df.to_csv("SentimentAnalysisFEB27.csv") 